In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import zipfile
import os
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from time import sleep
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import requests
from os import listdir
from os.path import isfile, join
import urllib
from urllib.request import urlretrieve 
from sentence_transformers import SentenceTransformer, util

import zipfile



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.model_selection import ParameterGrid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marcj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\marcj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [101]:
lista = [
"20ng.csv.zip", 
"ACM.csv.zip",
"CSTR.csv.zip",
"Dmoz-Business.csv.zip",
"Dmoz-Computers.csv.zip",
"Dmoz-Health.csv.zip",
"Dmoz-Science.csv.zip",
"Dmoz-Sports.csv.zip",
"Enron.csv.zip",
"Hitech.csv.zip",
"IrishEconomicSentiment.csv.zip",
"LATimes.csv.zip",
"NFS.csv.zip",
"Opinosis.csv.zip",
"PubMed_Cancer-2000.csv.zip",
"PubMed_Cancer.csv.zip",
"Re8.csv.zip",
"Reuters-21578.csv.zip",
"Reviews.csv.zip",
"multi-domain-sentiment.csv.zip",
"SpamAssassin.csv.zip",
"SyskillWebert.csv.zip",
"WebACE.csv.zip",
"fbis.csv.zip",
"industry-sector.csv.zip",
"la1.mat.csv.zip",
"la2.mat.csv.zip",
"novo3.mat.csv.zip",
"oh0.mat.csv.zip",
"oh10.mat.csv.zip",
"oh15.mat.csv.zip",
"oh5.mat.csv.zip",
"ohscal.mat.csv.zip",
"ohsumed-400.csv.zip",
"re0.mat.csv.zip",
"re1.mat.csv.zip",
"review_polarity.csv.zip",
"tr11.mat.csv.zip",
"tr12.mat.csv.zip",
"tr21.mat.csv.zip",
"tr23.mat.csv.zip",
"tr31.mat.csv.zip",
"tr41.mat.csv.zip",
"tr45.mat.csv.zip",
"trec7-3000.csv.zip",
"wap.mat.csv.zip",
"webkb.csv.zip"]

len(lista)


24

In [104]:

def dowload_data(lista):
    for arquivo in lista:
        url = f'https://github.com/ragero/text-collections/raw/master/Sequence_of_words_CSV/{arquivo}'
        #baixa o arquivo e salvar na pasta data utilizando a biblioteca requests
        r = requests.get(url)
        with open(f'data/{arquivo}', 'wb') as f:
            f.write(r.content)
"""        with zipfile.ZipFile(f'data/{arquivo}', 'r') as zip_ref:
            zip_ref.extractall('data')
        
        os.remove(f'data/{arquivo}')
        """

def extract_zip_files(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith('.zip'):
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(folder_path)
            os.remove(file_path)
        
dowload_data(lista)
extract_zip_files(path)

In [4]:
path = 'data/'

 
files =  [file for file in os.listdir(path) if file.endswith('.csv')]
#juntar todos os arquivos em um só
df = pd.concat([pd.read_csv(path + file) for file in files])

print(df)


                                                   Text           Class
0     subject manner similar termin graphic graphic ...  comp_windows_x
1     standard cost offic subject name count bruce v...  comp_windows_x
2     subject nasa nasa nasa nasa outland voic toolk...  comp_windows_x
3     info subject server share friend friend close ...  comp_windows_x
4     advantag helen helen code support subject citi...  comp_windows_x
...                                                 ...             ...
3895  jurisdict jurisdict jurisdict krakauer krakaue...         culture
3896  ulihrach ulihrach ulihrach cheer cheer constru...         culture
3897  oriol oriol affect affect confin confin tent t...         culture
3898  sofa sofa sofa sofa sofa sofa refrain refrain ...         culture
3899  titlehold titlehold verbal verbal construct co...         culture

[273175 rows x 2 columns]


In [5]:
def tokenization (df, column, new_column):
    df[column] = df[column].astype(str)
    df[new_column] = df[column].map(lambda x: word_tokenize(x))
    return df

In [11]:
df = tokenization(df, 'Text', 'tokens')
df.to_parquet('data/df.parquet')

In [12]:
df = pd.read_parquet('data/df.parquet')
df

,Text,Class,tokens
0,subject manner similar termin graphic graphic ...,comp_windows_x,"[subject, manner, similar, termin, graphic, gr..."
1,standard cost offic subject name count bruce v...,comp_windows_x,"[standard, cost, offic, subject, name, count, ..."
2,subject nasa nasa nasa nasa outland voic toolk...,comp_windows_x,"[subject, nasa, nasa, nasa, nasa, outland, voi..."
3,info subject server share friend friend close ...,comp_windows_x,"[info, subject, server, share, friend, friend,..."
4,advantag helen helen code support subject citi...,comp_windows_x,"[advantag, helen, helen, code, support, subjec..."
...,...,...,...
3895,jurisdict jurisdict jurisdict krakauer krakaue...,culture,"[jurisdict, jurisdict, jurisdict, krakauer, kr..."
3896,ulihrach ulihrach ulihrach cheer cheer constru...,culture,"[ulihrach, ulihrach, ulihrach, cheer, cheer, c..."
3897,oriol oriol affect affect confin confin tent t...,culture,"[oriol, oriol, affect, affect, confin, confin,..."
3898,sofa sofa sofa sofa sofa sofa refrain refrain ...,culture,"[sofa, sofa, sofa, sofa, sofa, sofa, refrain, ..."


In [14]:
print(type(df['Class']))

<class 'pandas.core.series.Series'>


In [15]:
le = LabelEncoder()
Y = le.fit_transform(df['Class'])

In [16]:
le.classes_

array(['100', '108', '11', '111', '118', '119', '12', '142', '161', '187',
       '189', '202', '221', '240', '3', '3DTech', '4', '95', 'Accounting',
       'AdHoc', 'Addictions', 'Aerospace_and_Defense', 'Agriculture',
       'Agriculture_and_Forestry', 'Alternative', 'Animal',
       'ArchitectureEducation', 'Artificial_Intelligence',
       'ArtificiallIntelligence', 'Arts_and_Entertainment', 'Astronomy',
       'Automotive', 'Bands', 'Baseball', 'Basketball', 'BioMedical',
       'Biology', 'Biotechnology_and_Pharmaceuticals', 'Bowling',
       'Business_Services', 'CAD_and_CAM', 'Chemicals', 'Chemistry',
       'CommunicationSystems', 'Companies', 'ComputationalGeometry',
       'Computer_Science', 'Conditions_and_Diseases',
       'Construction_and_Maintenance', 'Consultants',
       'Consumer_Goods_and_Services', 'Cricket', 'Cycling',
       'DataManagement', 'DataMining', 'Data_Communications',
       'Data_Formats', 'DatabaseSystems', 'DeclarativeProgramming',
       'Distribu

In [17]:
x_train, x_test, y_train, y_test = train_test_split(df['tokens'], Y, test_size=0.2, random_state=5)

In [18]:
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('clf', LinearSVC())
])
pipeline.steps

[('vectorizer', TfidfVectorizer()), ('clf', LinearSVC())]

In [ ]:
pipeline.fit(df['tokens'], Y)

In [55]:
pred = pipeline.predict(x_test)
pred

array([1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 1, 1, 0, 1, 1, 0, 0,
       2, 2, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 3,
       0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [56]:
y_test

array([1, 0, 3, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 1, 1, 0, 1, 1, 1, 0,
       2, 2, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 3, 2, 0, 3,
       0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [57]:
accuracy_score(y_test, pred)

0.8166666666666667

In [58]:
print(classification_report(y_test, pred, target_names=le.classes_))

                         precision    recall  f1-score   support

ArtificiallIntelligence       0.69      1.00      0.82        25
               Robotics       1.00      0.65      0.79        26
                Systems       1.00      0.80      0.89         5
                 Theory       1.00      0.75      0.86         4

               accuracy                           0.82        60
              macro avg       0.92      0.80      0.84        60
           weighted avg       0.87      0.82      0.82        60



In [64]:
#algo sobre artificial intelligence
text = ['sdsdsdsds'] 
pred = pipeline.predict(text)
pred

array([0])